In [1]:
'''
!pip install deepcut sklearn-crfsuite nltk
!pip install https://github.com/PyThaiNLP/pythainlp/archive/dev.zip
!wget https://www.dropbox.com/s/u1y5nl76mzwzdku/thai_ner-24072018-14-08-train.txt
'''

'\n!pip install deepcut sklearn-crfsuite nltk\n!pip install https://github.com/PyThaiNLP/pythainlp/archive/dev.zip\n!wget https://www.dropbox.com/s/u1y5nl76mzwzdku/thai_ner-24072018-14-08-train.txt\n'

In [2]:
# -*- coding: utf-8 -*-
# เรียกใช้งานโมดูล
file_name="data"
import codecs
from pythainlp.tokenize import word_tokenize
import deepcut
from pythainlp.tag import pos_tag
from nltk.tokenize import RegexpTokenizer
import glob
import nltk
import re
# thai cut
thaicut="deepcut"
# เตรียมตัวตัด tag ด้วย re
pattern = r'\[(.*?)\](.*?)\[\/(.*?)\]'
tokenizer = RegexpTokenizer(pattern) # ใช้ nltk.tokenize.RegexpTokenizer เพื่อตัด [TIME]8.00[/TIME] ให้เป็น ('TIME','ไง','TIME')
# จัดการกับ tag ที่ไม่ได้ tag
def toolner_to_tag(text):
 text=text.strip().replace("FACILITY","LOCATION").replace("[AGO]","").replace("[/AGO]","")
 text=re.sub("<[^>]*>","",text)
 text=re.sub("(\[\/(.*?)\])","\\1***",text)#.replace('(\[(.*?)\])','***\\1')# text.replace('>','>***') # ตัดการกับพวกไม่มี tag word
 text=re.sub("(\[\w+\])","***\\1",text)
 text2=[]
 for i in text.split('***'):
  if "[" in i:
   text2.append(i)
  else:
   text2.append("[word]"+i+"[/word]")
 text="".join(text2)#re.sub("[word][/word]","","".join(text2))
 return text.replace("[word][/word]","")
# แปลง text ให้เป็น conll2002
def text2conll2002(text,pos=True):
    """
    ใช้แปลงข้อความให้กลายเป็น conll2002
    """
    text=toolner_to_tag(text)
    text=text.replace("''",'"')
    text=text.replace("’",'"').replace("‘",'"')#.replace('"',"")
    tag=tokenizer.tokenize(text)
    j=0
    conll2002=""
    for tagopen,text,tagclose in tag:
        word_cut=deepcut.tokenize(text)#word_tokenize(text,engine=thaicut) # ใช้ตัวตัดคำ newmm
        i=0
        txt5=""
        while i<len(word_cut):
            if word_cut[i]=="''" or word_cut[i]=='"':pass
            elif i==0 and tagopen!='word':
                txt5+=word_cut[i]
                txt5+='\t'+'B-'+tagopen
            elif tagopen!='word':
                txt5+=word_cut[i]
                txt5+='\t'+'I-'+tagopen
            else:
                txt5+=word_cut[i]
                txt5+='\t'+'O'
            txt5+='\n'
            #j+=1
            i+=1
        conll2002+=txt5
    if pos==False:
        return conll2002
    return postag(conll2002)
# ใช้สำหรับกำกับ pos tag เพื่อใช้กับ NER
# print(text2conll2002(t,pos=False))
def postag(text):
    listtxt=[i for i in text.split('\n') if i!='']
    list_word=[]
    for data in listtxt:
        list_word.append(data.split('\t')[0])
    #print(text)
    list_word=pos_tag(list_word,engine='perceptron')
    text=""
    i=0
    for data in listtxt:
        text+=data.split('\t')[0]+'\t'+list_word[i][1]+'\t'+data.split('\t')[1]+'\n'
        i+=1
    return text
# เขียนไฟล์ข้อมูล conll2002
def write_conll2002(file_name,data):
    """
    ใช้สำหรับเขียนไฟล์
    """
    with codecs.open(file_name, "w", "utf-8-sig") as temp:
        temp.write(data)
    return True
# อ่านข้อมูลจากไฟล์
def get_data(fileopen):
	"""
    สำหรับใช้อ่านทั้งหมดทั้งในไฟล์ทีละรรทัดออกมาเป็น list
    """
	with codecs.open(fileopen, 'r',encoding='utf-8-sig') as f:
		lines = f.read().splitlines()
	return lines

def alldata(lists):
    text=""
    for data in lists:
        text+=text2conll2002(data)
        text+='\n'
    return text

def alldata_list(lists):
    data_all=[]
    for data in lists:
        data_num=[]
        try:
            txt=text2conll2002(data,pos=True).split('\n')
            for d in txt:
                tt=d.split('\t')
                if d!="":
                    if len(tt)==3:
                        data_num.append((tt[0],tt[1],tt[2]))
                    else:
                        data_num.append((tt[0],tt[1]))
            #print(data_num)
            data_all.append(data_num)
        except:
            print(data)
    #print(data_all)
    return data_all

def alldata_list_str(lists):
	string=""
	for data in lists:
		string1=""
		for j in data:
			string1+=j[0]+"	"+j[1]+"	"+j[2]+"\n"
		string1+="\n"
		string+=string1
	return string

def get_data_tag(listd):
	list_all=[]
	c=[]
	for i in listd:
		if i !='':
			c.append((i.split("\t")[0],i.split("\t")[1],i.split("\t")[2]))
		else:
			list_all.append(c)
			c=[]
	return list_all
def getall(lista):
    ll=[]
    for i in lista:
        o=True
        for j in ll:
            if re.sub("\[(.*?)\]","",i)==re.sub("\[(.*?)\]","",j):
                o=False
                break
        if o==True:
            ll.append(i)
    return ll

data1=getall(get_data(file_name+".txt"))
datatofile=alldata_list(data1)
tt=[]

Using TensorFlow backend.
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
'''import codecs
with codecs.open("thainlp-28072018-pos.conll", 'r',encoding='utf-8-sig') as f:
    t= f.read().split("\n\n")
datatofile =[[(i.split("\t")[0].strip(),i.split("\t")[1].strip(),i.split("\t")[2].strip()) for i in tt.splitlines()] for tt in t]
'''

'import codecs\nwith codecs.open("thainlp-28072018-pos.conll", \'r\',encoding=\'utf-8-sig\') as f:\n    t= f.read().split("\n\n")\ndatatofile =[[(i.split("\t")[0].strip(),i.split("\t")[1].strip(),i.split("\t")[2].strip()) for i in tt.splitlines()] for tt in t]\n'

In [4]:
print(datatofile[1])

[('สงสัย', 'NCMN', 'O'), ('ติด', 'VACT', 'O'), ('หวัด', 'NCMN', 'O'), ('นก', 'DDAC', 'O'), (' ', 'NCMN', 'O'), ('อีก', 'DDBQ', 'O'), ('คน', 'NCMN', 'O'), ('ยัง', 'XVBM', 'O'), ('น่า', 'VSTA', 'O'), ('ห่วง', 'NCMN', 'O')]


## สร้างไฟล์ conll

In [5]:
'''with open(file_name+"-pos.conll","w") as f:
    i=0
    while i<len(datatofile):
        for j in datatofile[i]:
            f.write(j[0]+"\t"+j[1]+"\t"+j[2]+"\n")
        if i+1<len(datatofile):
            f.write("\n")
        i+=1'''

'with open(file_name+"-pos.conll","w") as f:\n    i=0\n    while i<len(datatofile):\n        for j in datatofile[i]:\n            f.write(j[0]+"\t"+j[1]+"\t"+j[2]+"\n")\n        if i+1<len(datatofile):\n            f.write("\n")\n        i+=1'

In [6]:
'''
class TrainChunker(nltk.ChunkParserI):
    """
    ใช้ในการ Train และรัน
    """
    def __init__(self, train_sents,testdata):
        train_data = [[(t,c) for w,t,c in sent] for sent in train_sents]
        test_data = [[(t,c) for w,t,c in sent] for sent in testdata]
        self.tagger = nltk.UnigramTagger(train_data)#nltk.NaiveBayesClassifier.train(train_data)
        self.tagger = nltk.tag.BigramTagger(train_data, backoff=self.tagger)
        self.tagger = nltk.tag.TrigramTagger(train_data, backoff=self.tagger)
        print(self.tagger.evaluate(test_data))
    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)#classify(pos_tags)#
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word.replace('<space>',' '), pos, chunktag) for ((word,pos),chunktag) in zip(sentence, chunktags)]
        #print(conlltags)
        return conlltags

def run(lists,test):
    """
    ใช้ในการทดสอบ NER
    """
    data=lists
    tag=TrainChunker(data,test)
    while True:
        texts=input("Text : ")
        toword=word_tokenize(texts,engine=thaicut)
        pos=pos_tag(toword,engine='perceptron')
        ner=tag.parse(pos)
        print([(word, chunktag) for (word,pos,chunktag) in ner])
'''

'\nclass TrainChunker(nltk.ChunkParserI):\n    """\n    ใช้ในการ Train และรัน\n    """\n    def __init__(self, train_sents,testdata):\n        train_data = [[(t,c) for w,t,c in sent] for sent in train_sents]\n        test_data = [[(t,c) for w,t,c in sent] for sent in testdata]\n        self.tagger = nltk.UnigramTagger(train_data)#nltk.NaiveBayesClassifier.train(train_data)\n        self.tagger = nltk.tag.BigramTagger(train_data, backoff=self.tagger)\n        self.tagger = nltk.tag.TrigramTagger(train_data, backoff=self.tagger)\n        print(self.tagger.evaluate(test_data))\n    def parse(self, sentence):\n        pos_tags = [pos for (word,pos) in sentence]\n        tagged_pos_tags = self.tagger.tag(pos_tags)#classify(pos_tags)#\n        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]\n        conlltags = [(word.replace(\'<space>\',\' \'), pos, chunktag) for ((word,pos),chunktag) in zip(sentence, chunktags)]\n        #print(conlltags)\n        return conlltags\n\ndef run(

## เตรียมส่วน train และ test

In [7]:
import random
#random.shuffle(datatofile)
print(len(datatofile))
training_samples = datatofile[:int(len(datatofile) * 0.8)]
test_samples = datatofile[int(len(datatofile) * 0.8):]
'''training_samples = datatofile[:2822]
test_samples = datatofile[2822:]'''
print(test_samples[0])
#tag=TrainChunker(training_samples,test_samples) # Train

#run(training_samples,test_samples)

4334
[('สถาบัน', 'NCMN', 'B-ORGANIZATION'), ('โรคทรวงอก', 'NCMN', 'I-ORGANIZATION'), (' ', 'NCMN', 'O'), ('กรมการแพทย์', 'NCMN', 'B-ORGANIZATION'), (' ', 'NCMN', 'O'), ('แนะ', 'VACT', 'O'), ('อาหาร', 'NCMN', 'O'), ('ควบคุม', 'VACT', 'O'), ('เบาหวาน', 'NCMN', 'O'), (' ', 'NCMN', 'O'), ('ควร', 'XVMM', 'O'), ('เลือก', 'VACT', 'O'), ('บริโภค', 'NCMN', 'O'), ('และ', 'JCRG', 'O'), ('ปฏิบัติ', 'VACT', 'O'), ('อย่าง', 'FIXV', 'O'), ('ไร', 'VSTA', 'O'), (' ', 'NCMN', 'O'), ('ให้', 'JSBR', 'O'), ('คุม', 'NCMN', 'O'), ('เบา', 'VATT', 'O'), ('หวาน', 'NCMN', 'O'), ('ได้', 'XVAE', 'O'), (' ', 'NCMN', 'O'), ('https', 'NCMN', 'B-URL'), (':', 'NCMN', 'I-URL'), ('/', 'NCMN', 'I-URL'), ('/goo.gl/VubCEF', 'NCMN', 'I-URL')]


In [8]:
t=0
te=0
for i in training_samples:
    for j in i:
        t+=1
for i in test_samples:
    for j in i:
        te+=1
print("จำนวนคำที่ใช้ train : "+str(t))
print("จำนวนคำที่ใช้ test : "+str(te))
print("จำนวนคำทั้งหมด : "+str(te+t))

จำนวนคำที่ใช้ train : 158340
จำนวนคำที่ใช้ test : 27401
จำนวนคำทั้งหมด : 185741


In [9]:
ct=0
cte=0
a=[]
b=[]
for i in training_samples:
    for j in i:
        if 'B-' in j[2]:
            ct+=1
            a.append(j[2])
for i in test_samples:
    for j in i:
        if 'B-' in j[2]:
            cte+=1
            b.append(j[2])

print("จำนวน tag ที่ใช้ train : "+str(ct))
print("จำนวน tag ที่ใช้ test : "+str(cte))
print("จำนวน tag ทั้งหมด : "+str(cte+ct))

จำนวน tag ที่ใช้ train : 9291
จำนวน tag ที่ใช้ test : 2311
จำนวน tag ทั้งหมด : 11602


In [10]:
from collections import Counter
print(Counter(a))
print(Counter(b))

Counter({'B-LOCATION': 2710, 'B-ORGANIZATION': 2462, 'B-PERSON': 1972, 'B-DATE': 973, 'B-TIME': 623, 'B-MONEY': 239, 'B-PERCENT': 86, 'B-LEN': 77, 'B-PHONE': 72, 'B-URL': 60, 'B-ZIP': 12, 'B-EMAIL': 5})
Counter({'B-ORGANIZATION': 867, 'B-LOCATION': 674, 'B-PERSON': 242, 'B-DATE': 237, 'B-MONEY': 107, 'B-TIME': 63, 'B-PERCENT': 37, 'B-PHONE': 27, 'B-URL': 25, 'B-LEN': 13, 'B-ZIP': 13, 'B-EMAIL': 5, 'B- ร่วมประชุมทางไกลกับผู้อำนวยการ[ORGANIZATION': 1})


## เตรียมเครื่องมือ CRF

In [11]:
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import sklearn_crfsuite

In [12]:
from pythainlp.corpus import stopwords
stopwords = stopwords.words('thai')

# แบบใช้ POS Tag

## เตรียมข้อมูล

In [13]:
def is_stopword(word):
    return word in stopwords
def is_s(word):
    if word == " " or word =="\t" or word=="":
        return True
    else:
        return False
def doc2features0(doc, i):
    word = doc[i][0]
    postag = doc[i][1]
    # Features from current word
    features={
        'word.word': word,
        'word.stopword': is_stopword(word),
        'postag':postag,
        'postag[:2]': postag[:2],
        'word.isdigit()': word.isdigit(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:]
    }
    # Features from previous word
    if i > 0:
        prevword = doc[i-1][0]
        postag1 = doc[i-1][1]
        features['word.prevword'] = prevword
        features['word.prepostag'] = postag1
        features['-1:postag[:2]']: postag1[:2]
        features['word.prevwordisdigit'] = prevword.isdigit()
    else:
        features['BOS'] = True # Special "Beginning of Sequence" tag
    # Features from next word
    if i < len(doc)-1:
        nextword = doc[i+1][0]
        postag1 = doc[i+1][1]
        features['word.nextword'] = nextword
        features['word.nextpostag'] = postag1
        features['word.nextwordisdigit'] = nextword.isdigit()
        features['+1:postag[:2]']: postag1[:2]
    else:
        features['EOS'] = True # Special "End of Sequence" tag
    return features

def extract_features0(doc):
    return [doc2features0(doc, i) for i in range(len(doc))]

def get_labels(doc):
    return [tag for (token,postag,tag) in doc]

X = [extract_features0(doc) for doc in training_samples]
y = [get_labels(doc) for doc in training_samples]

X_test = [extract_features0(doc) for doc in test_samples]
y_test = [get_labels(doc) for doc in test_samples]
#print(X[0])
print(len(training_samples))

3467


In [14]:
def is_stopword(word):
    return word in stopwords
def doc2features2(doc, i):
    word = doc[i]
    postag = doc[i][1]
    # Features from current word
    features={
        'word.word': word,
        'word.stopword': is_stopword(word),
        'postag':postag,
        'postag[:2]': postag[:2],
        'word.isdigit()': word.isdigit(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:]
    }
    # Features from previous word
    if i > 0:
        prevword = doc[i-1]
        postag1 = doc[i-1][1]
        features['word.prevword'] = prevword
        features['word.prepostag'] = doc[i-1][1]
        features['-1:postag[:2]']: postag1[:2]
        features['word.prevwordisdigit'] = prevword.isdigit()
    else:
        features['BOS'] = True # Special "Beginning of Sequence" tag
    # Features from next word
    if i < len(doc)-1:
        nextword = doc[i+1]
        postag1 = doc[i+1][1]
        features['word.nextword'] = nextword
        features['+1:postag[:2]']: postag1[:2]
        features['-1:postag[:2]']: postag1[:2]
        features['word.nextwordisdigit'] = nextword.isdigit()
    else:
        features['EOS'] = True # Special "End of Sequence" tag
    return features

def extract_features2(tag):
    i=0
    l=[]
    while i<len(tag):
        l.append(doc2features2(tag,i))
        i+=1
    return l

## Train

In [15]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=500,
    all_possible_transitions=True,
    model_filename=file_name+"-pos-new.model2"
)
crf.fit(X, y);

#X_test = extract_features2(["พวกเรา","จะ","ไป","เที่ยว","หนองคาย"])
#print(crf.predict_single(X_test))

CPU times: user 4min 46s, sys: 54.2 ms, total: 4min 46s
Wall time: 4min 46s


## Test

In [16]:
labels = list(crf.classes_)
labels.remove('O')
y_pred = crf.predict(X_test)
e=metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)
print(e)
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

0.7333225049665952
                precision    recall  f1-score   support

        B-DATE      0.858     0.869     0.864       237
        I-DATE      0.923     0.939     0.931       623
       B-EMAIL      0.667     0.800     0.727         5
       I-EMAIL      0.000     0.000     0.000         0
         B-LEN      1.000     0.615     0.762        13
         I-LEN      0.917     0.564     0.698        39
    B-LOCATION      0.791     0.579     0.668       674
    I-LOCATION      0.795     0.455     0.579       290
       B-MONEY      0.947     0.832     0.886       107
       I-MONEY      0.953     0.874     0.912       444
B-ORGANIZATION      0.902     0.630     0.742       867
I-ORGANIZATION      0.464     0.277     0.347       350
     B-PERCENT      0.974     1.000     0.987        37
     I-PERCENT      0.915     1.000     0.955       107
      B-PERSON      0.681     0.521     0.590       242
      I-PERSON      0.760     0.244     0.369        78
       B-PHONE      0.917   

/home/wannaphong/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/wannaphong/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/wannaphong/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/wannaphong/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [17]:
import eli5
eli5.show_weights(crf, top=30)

From \ To,O,B-DATE,I-DATE,B-EMAIL,I-EMAIL,B-LEN,I-LEN,B-LOCATION,I-LOCATION,B-MONEY,I-MONEY,B-ORGANIZATION,I-ORGANIZATION,B-PERCENT,I-PERCENT,B-PERSON,I-PERSON,B-PHONE,I-PHONE,B-TIME,I-TIME,B-URL,I-URL,B-ZIP
O,5.231,1.85,-4.935,0.135,-0.751,0.627,-2.799,3.008,-3.929,1.492,-3.262,3.018,-4.425,1.501,-3.438,3.087,-3.65,0.438,-2.502,1.989,-4.468,0.292,-1.951,0.0
B-DATE,-0.451,-0.256,6.719,0.0,0.0,0.0,-0.496,-0.28,0.0,0.0,-0.681,-0.21,-0.095,0.0,-0.316,-0.36,0.0,0.0,-0.567,0.0,-1.132,0.0,0.0,0.0
I-DATE,0.016,-1.595,5.724,0.0,0.0,0.0,-0.257,-1.067,-0.778,0.0,-0.451,-0.006,-0.733,0.0,-0.279,-0.42,-0.58,0.0,-0.549,0.0,-1.77,-0.054,0.0,0.0
B-EMAIL,-1.336,0.0,0.0,0.0,1.378,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-EMAIL,-0.296,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-LEN,-2.554,0.0,-0.094,0.0,0.0,0.0,6.08,0.0,0.0,0.0,-0.159,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.48,0.0,0.0,0.0
I-LEN,-0.488,0.0,-0.327,0.0,0.0,0.0,6.26,0.0,0.0,0.0,-0.019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.908,-0.293,0.0,0.0,0.0
B-LOCATION,1.505,-1.043,-0.657,0.0,0.0,0.0,0.0,2.572,6.113,0.0,0.0,-0.0,-0.767,-0.016,0.0,1.041,-0.34,0.0,0.0,-0.298,-0.217,0.0,0.0,0.0
I-LOCATION,0.197,-0.599,-0.934,0.0,0.0,-0.095,-0.724,0.792,6.876,0.0,-0.123,0.0,-0.68,-0.734,-0.058,-0.644,-0.386,0.0,-0.122,-0.22,-0.55,0.0,0.0,-0.184
B-MONEY,-0.981,0.0,-0.064,0.0,0.0,0.0,-0.361,0.0,0.0,0.0,6.944,0.0,0.0,0.0,-0.516,0.0,0.0,0.0,-0.369,0.0,-0.833,0.0,0.0,0.0


In [18]:
"""
[[('เจ้าของ', 'NCMN', 'O'), ('ไก่', 'NCMN', 'O'), ('ชน', 'NCMN', 'O'), ('ฆ่า', 'NCMN', 'O'), ('ไก่', 'NCMN', 'O'), ('ประชด', 'NCMN', 'O')]]
"""
def get_ner(text):
    word_cut=word_tokenize(text,engine=thaicut)
    list_word=pos_tag(word_cut,engine='perceptron')
    #print(list_word)
    X_test = extract_features0([(data,list_word[i][1]) for i,data in enumerate(word_cut)])
    y_=crf.predict_single(X_test)
    return [(word_cut[i],list_word[i][1],data) for i,data in enumerate(y_)]

In [19]:
get_ner("บ่ายสองโมงนี้ทำอะไรดี")

[('บ่าย', 'NCMN', 'B-TIME'),
 ('สอง', 'DCNM', 'I-TIME'),
 ('โมง', 'CNIT', 'I-TIME'),
 ('นี้', 'DDAC', 'I-TIME'),
 ('ทำ', 'VACT', 'O'),
 ('อะไร', 'PDMN', 'O'),
 ('ดี', 'ADVN', 'O')]

In [20]:
get_ner("นางสาวพิมพ์ผกา ผู้ยิ้ม กำลังเดินทางไปรับตำแหน่งครูที่โรงเรียนนางรอง")

[('นางสาวพิมพ์', 'NCMN', 'O'),
 ('ผกา', 'NCMN', 'O'),
 (' ', 'NCMN', 'O'),
 ('ผู้', 'PREL', 'O'),
 ('ยิ้ม', 'VACT', 'O'),
 (' ', 'NCMN', 'O'),
 ('กำลัง', 'XVBM', 'O'),
 ('เดินทาง', 'VACT', 'O'),
 ('ไป', 'XVAE', 'O'),
 ('รับ', 'VACT', 'O'),
 ('ตำแหน่ง', 'NCMN', 'O'),
 ('ครู', 'NCMN', 'O'),
 ('ที่', 'PREL', 'O'),
 ('โรง', 'NCMN', 'B-LOCATION'),
 ('เรียนนางรอง', 'VACT', 'I-LOCATION')]

# แบบไม่ใช้ POS Tag

## เอา POS Tag ออก

In [21]:
training_samples_notpost=[]
for i in training_samples:
    t=[]
    for j in i:
        t.append((j[0],j[2]))
    training_samples_notpost.append(t)

In [22]:
training_samples[0]

[('http://www.bangkokhealth.com/healthnews_htdoc/healthnews _ detail.asp?Number=10506',
  'NCMN',
  'B-URL')]

In [23]:
training_samples_notpost[0]

[('http://www.bangkokhealth.com/healthnews_htdoc/healthnews _ detail.asp?Number=10506',
  'B-URL')]

In [24]:
test_samples_notpost=[]
for i in test_samples:
    t=[]
    for j in i:
        t.append((j[0],j[2]))
    test_samples_notpost.append(t)

## เตรียมข้อมูล train และ test

In [25]:
def is_stopword(word):
    return word in stopwords
def is_s(word):
    if word == " " or word =="\t" or word=="":
        return True
    else:
        return False
def doc2features_notpost(doc, i):
    word = doc[i][0]
    # Features from current word
    features={
        'bias': 1.0,
        'word.word': word,
        'word.stopword': is_stopword(word),
        'word.isdigit()': word.isdigit(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:]
    }
    # Features from previous word
    if i > 0:
        prevword = doc[i-1][0]
        features['word.prevword'] = prevword
        features['word.prevwordisdigit'] = prevword.isdigit()
    else:
        features['BOS'] = True # Special "Beginning of Sequence" tag
    # Features from next word
    if i < len(doc)-1:
        nextword = doc[i+1][0]
        features['word.nextword'] = nextword
        features['word.nextwordisdigit'] = nextword.isdigit()
    else:
        features['EOS'] = True # Special "End of Sequence" tag
    return features

def extract_features_notpost(doc):
    return [doc2features_notpost(doc, i) for i in range(len(doc))]

def get_labels(doc):
    return [tag for (token,tag) in doc]

X = [extract_features_notpost(doc) for doc in training_samples_notpost]#datatofile]#
y = [get_labels(doc) for doc in training_samples_notpost]#datatofile]#

X_test = [extract_features_notpost(doc) for doc in test_samples_notpost]
y_test = [get_labels(doc) for doc in test_samples_notpost]

In [26]:
def is_stopword(word):
    return word in stopwords
def doc2features2_notpost(doc, i):
    word = doc[i]
    # Features from current word
    features={
        'bias': 1.0,
        'word.word': word,
        'word.stopword': is_stopword(word),
        'word.isdigit()': word.isdigit(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:]
    }
    # Features from previous word
    if i > 0:
        prevword = doc[i-1]
        features['word.prevword'] = prevword
        features['word.prevwordisdigit'] = prevword.isdigit()
    else:
        features['BOS'] = True # Special "Beginning of Sequence" tag
    # Features from next word
    if i < len(doc)-1:
        nextword = doc[i+1]
        features['word.nextword'] = nextword
        features['word.nextwordisdigit'] = nextword.isdigit()
    else:
        features['EOS'] = True # Special "End of Sequence" tag
    return features

def extract_features2_notpost(tag):
    i=0
    l=[]
    while i<len(tag):
        l.append(doc2features2_notpost(tag,i))
        i+=1
    return l

## Train

In [27]:
%%time
crf2 = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=600,
    all_possible_transitions=True,
    model_filename=file_name+"-notpostag-new.model2"
)
crf2.fit(X, y);

CPU times: user 6min 22s, sys: 42.8 ms, total: 6min 22s
Wall time: 6min 22s


## Test

In [28]:
labels = list(crf2.classes_)
labels.remove('O')
y_pred = crf2.predict(X_test)
e=metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)
print(e)
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))
X_test = extract_features2(["พวกเรา","จะ","ไป","เที่ยว","หนองคาย"])
print(crf2.predict_single(X_test))

0.7320200711076247
                precision    recall  f1-score   support

        B-DATE      0.855     0.844     0.849       237
        I-DATE      0.923     0.921     0.922       623
       B-EMAIL      0.667     0.800     0.727         5
       I-EMAIL      0.000     0.000     0.000         0
         B-LEN      1.000     0.692     0.818        13
         I-LEN      0.923     0.615     0.738        39
    B-LOCATION      0.781     0.576     0.663       674
    I-LOCATION      0.737     0.434     0.547       290
       B-MONEY      0.936     0.822     0.876       107
       I-MONEY      0.959     0.892     0.924       444
B-ORGANIZATION      0.904     0.633     0.745       867
I-ORGANIZATION      0.476     0.257     0.334       350
     B-PERCENT      0.973     0.973     0.973        37
     I-PERCENT      0.920     0.963     0.941       107
      B-PERSON      0.665     0.517     0.581       242
      I-PERSON      0.690     0.256     0.374        78
       B-PHONE      0.815   

/home/wannaphong/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/wannaphong/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/wannaphong/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/wannaphong/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [29]:
with open(file_name+".conll","w") as f:
    i=0
    while i<len(datatofile):
        for j in datatofile[i]:
            f.write(j[0]+"\t"+j[1]+"\t"+j[2]+"\n")
        if i+1<len(datatofile):
            f.write("\n")
        i+=1

In [30]:
with open(file_name+"-test20-nopostag.conll","w") as f:
    i=0
    while i<len(test_samples_notpost):
        for j in test_samples_notpost[i]:
            f.write(j[0]+"\t"+j[1]+"\n")
        if i+1<len(test_samples_notpost):
            f.write("\n")
        i+=1